In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD

connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
%cd ../drive/My Drive/bigdatahw4

download the kaggle dataset to google drive

In [ ]:
#insert the kaggle json file in the content directory before running this cell
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

In [ ]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('amazon-books-reviews.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

read the dataset or just a portion for testing purposes

In [ ]:

percentage=1
nRows=3000001
nOfRowsToRead=int(nRows*percentage)

#read the csv text file
df = pd.read_csv('./Books_rating.csv', header=None , nrows=nOfRowsToRead)
df=df.to_numpy()

#split the data into header and data and reviews
header= df[0]
data=df[1:]
docs=data[:,-1]

print(docs.shape)

In [ ]:
#drop nan from the rows to not have problems
toDrop=[i for i,d in enumerate(docs) if type(docs[i])!=str]
docs=np.delete(docs,toDrop)

preprocess the data with tfidfVectorizer

In [ ]:
#lowercase the reviews
for i in range(len(docs)):
  docs[i] = docs[i].lower()

print("lowercase done")

#remove non alphabetic characters
for i in range(len(docs)):
  docs[i] = re.sub(r"[^a-zA-Z'\n ]", "", docs[i])

print("remove non alphabetic characters done")

#remove contractions
contractions_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

# Regular expression for finding contractions
def multiple_replace(dict, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

docs = [multiple_replace(contractions_dict, doc) for doc in docs] # Removing contractions

print("contractions done")

#remove \n
docs = [re.sub(r"[^a-zA-Z ]", " ", doc) for doc in docs]

#remove stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

stopwords = nltk.corpus.stopwords.words('english') # Returns a list
stopwords = set(stopwords) # We want a set, because this is implemented with a hash table
                           # Checking the if condition in rem_stop costs O(1) in this way

def rem_stop(doc):
   word_list = word_tokenize(doc)
   cleaned_doc = ""
   for word in word_list:
     if word not in stopwords:
       cleaned_doc += " " + word
   return cleaned_doc


docs = [rem_stop(doc) for doc in docs]

print("stopwords done")

#lemmatiation
lemmatizer = WordNetLemmatizer()
for i in range(len(docs)):
    word_list = word_tokenize(docs[i])
    lemmatized_doc = ""
    for word in word_list:
        lemmatized_doc = lemmatized_doc + " " + lemmatizer.lemmatize(word)
    docs[i] = lemmatized_doc  

print("lemmetizer done")

#tfidf vectorization
vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english', min_df=5) ## Corpus is in English
X = vectorizer.fit_transform(docs)

print("vectorizer done")

print(X.shape)

In [ ]:
# write X to a file
with open('X.pickle', 'wb') as f:
    pickle.dump(X, f)
#save the fitted vectorizer to not redo the preprocessing again
with open('vectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)

read the preprocessed tfidf dataset written to save it

In [ ]:
#read X from a file
with open('X.pickle', 'rb') as f:
    X = pickle.load(f)
#read the vectorizer from a file
with open('vectorizer.pickle', 'rb') as f:
    vectorizer = pickle.load(f)
feature_names=vectorizer.get_feature_names_out(input_features=X)

In [ ]:
#take just a percantage of it for testing 
percentage=0.001
rowsToRetain=int(X.shape[0]*percentage)
X=X[:rowsToRetain,:]
X.shape

apply truncated svd

In [ ]:
k=3
svd = TruncatedSVD(n_components=k ,random_state=42)
y=svd.fit_transform(X)
svd.explained_variance_ratio_,y.shape,svd.explained_variance_ratio_.sum()

In [ ]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    s = ""
    for t in sorted_terms:
        s += t[0] + " "
    print(s)

apply k means

In [ ]:
k=20
km = KMeans(n_clusters=k, init='k-means++', max_iter=100)
km.fit(X)

In [ ]:
centroids = km.cluster_centers_.argsort()[:, ::-1] ## Indices of largest centroids' entries in descending order
terms = vectorizer.get_feature_names()
for i in range(k):
    print("Cluster %d:" % i, end='')
    for ind in centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()